In [79]:
import numpy as np
import pandas as pd

from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC

from nltk.corpus import stopwords

In [2]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [3]:
neg_rev = [movie_reviews.words(fileids=[f]) for f in negids]
pos_rev = [movie_reviews.words(fileids=[f]) for f in posids]

In [4]:
print('Negative Review Shape: ', len(neg_rev))
print('Positive Review Shape: ', len(pos_rev))

Negative Review Shape:  1000
Positive Review Shape:  1000


Каждый объект имеет тип:

In [5]:
type(pos_rev[0])

nltk.corpus.reader.util.StreamBackedCorpusView

Далее, чтобы использовать `CountVectorizer()` представим каждый текст строкой. 

Таким образом мы получим список строк, где каждая строка это текст.

In [24]:
neg_rev = [' '.join(movie_reviews.words(fileids=f)) for f in negids]
pos_rev = [' '.join(movie_reviews.words(fileids=f)) for f in posids]

Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.

In [25]:
all_reviews = neg_rev + pos_rev
labels = np.array([0 for _ in range(1000)] + [1 for _ in range(1000)])

Подсчитайте количество отзывов в выборке.

In [26]:
print('N Reviews: ', len(all_reviews))

N Reviews:  2000


Подсчитайте долю класса 1 в выборке

In [27]:
print('Positive Review Shape: ', len(pos_rev))

Positive Review Shape:  1000


Импортируйте `CountVectorizer` из `sklearn.feature_extraction.text` Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод `fit_transform` у `CountVectorizer` успешно отрабатывал. Подсчитайте количество признаков в `CountVectorizer`.

Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [28]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(all_reviews)

print('N Features: ', len(vectorizer.get_feature_names()))

N Features:  39659


Соберите `pipeline` из `CountVectorizer` и `LogisticRegression` c настройками по-умолчанию и с помощью `cross_val_score` (также со стандартными настройками) оцените получаемое "из коробки" качество по `accuracy`.

In [58]:
pipeline = Pipeline([
    ('feat_gen', CountVectorizer()),
    ('model', LogisticRegression(solver='liblinear'))
])

cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
print('Accuracy CV: ', np.mean(cv_results))

Accuracy CV:  0.841


Аналогично accuracy, оцените качество по `ROC-AUC`

In [59]:
cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='roc_auc', error_score='raise')
print('ROC-AUC CV: ', round(np.mean(cv_results),3))

ROC-AUC CV:  0.916


Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). 

Вам могут пригодиться метод `get_feature_names()` или поле `vocabulary_` у класса `CountVectorizer`

In [60]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(all_reviews)

model = LogisticRegression(solver='liblinear')
model.fit(X, labels)

LogisticRegression(solver='liblinear')

In [56]:
imp_dict = dict(zip(vectorizer.vocabulary_.keys(), model.coef_[0]))

feat_imp = pd.DataFrame(imp_dict, index=[0]).T

print('Top 5 Features:')
print(abs(feat_imp).sort_values(by=0, ascending=False).head())

Top 5 Features:
                 0
techie    0.782254
khe       0.636624
moff      0.592839
unsigned  0.556051
ankles    0.508191


In [57]:
top_feat = abs(feat_imp).sort_values(by=0, ascending=False).head().index
for feat in top_feat:
    print(f'{feat} freq: ', vectorizer.vocabulary_[feat])

techie freq:  35004
khe freq:  19411
moff freq:  22764
unsigned freq:  37323
ankles freq:  1896


In [34]:
print('Avg Word Feature Freq: ', np.array(list(vectorizer.vocabulary_.values())).mean())

Avg Word Feature Freq:  19829.0


Здесь и далее оценка качества будет выполняться с помощью `cross_val_score` с `cv=5` и остальными параметрами по умолчанию. Оцените среднее качество: `mean()` и стандартное отклонение `std()` по фолдамам для:
- а) `pipeline` из `CountVectorizer()` и `LogisticRegression()`
- б) `pipeline` из `TfidfVectorizer()` и `LogisticRegression()`

In [64]:
pipeline = Pipeline([
    ('feat_gen', CountVectorizer()),
    ('model', LogisticRegression(solver='liblinear'))
])

cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
print('Mean Accuracy CV : ', np.mean(cv_results))
print('Std Accuracy CV : ', np.std(cv_results))

Mean Accuracy CV :  0.841
Std Accuracy CV :  0.01677796173556255


In [68]:
pipeline = Pipeline([
    ('feat_gen', TfidfVectorizer()),
    ('model', LogisticRegression(solver='liblinear'))
])

cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
print('Mean Accuracy CV : ', np.mean(cv_results))
print('Std Accuracy CV : ', np.std(cv_results))

Mean Accuracy CV :  0.8210000000000001
Std Accuracy CV :  0.004062019202317978


Попробуйте задавать разные значения параметра `min_df` у `CountVectorizer`. 

Оцените качество вашего классификатора с `min_df=10` и с `min_df=50`

In [73]:
min_dfs = [10, 50]

for min_df in min_dfs:
    pipeline = Pipeline([
        ('feat_gen', CountVectorizer(min_df=min_df)),
        ('model', LogisticRegression(solver='liblinear'))
    ])
    
    print('\nmin_df: ', min_df)
    cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
    print('Mean Accuracy CV : ', np.mean(cv_results))
    print('Std Accuracy CV : ', np.std(cv_results))


min_df:  10
Mean Accuracy CV :  0.8390000000000001
Std Accuracy CV :  0.011895377253370336

min_df:  50
Mean Accuracy CV :  0.813
Std Accuracy CV :  0.013453624047073712


Попробуйте использовать разные классификаторы после `CountVectorizer`. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для `LogisticRegression`, `LinearSVC` и `SGDClassifier`.

Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

In [74]:
pipeline = Pipeline([
    ('feat_gen', CountVectorizer()),
    ('model', LogisticRegression())
])

cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
print('Mean Accuracy CV : ', np.mean(cv_results))
print('Std Accuracy CV : ', np.std(cv_results))

Mean Accuracy CV :  0.8424999999999999
Std Accuracy CV :  0.021794494717703363


In [76]:
pipeline = Pipeline([
    ('feat_gen', CountVectorizer()),
    ('model', LinearSVC())
])

cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
print('Mean Accuracy CV : ', np.mean(cv_results))
print('Std Accuracy CV : ', np.std(cv_results))

Mean Accuracy CV :  0.8325000000000001
Std Accuracy CV :  0.0162788205960997


In [78]:
pipeline = Pipeline([
    ('feat_gen', CountVectorizer()),
    ('model', SGDClassifier())
])

cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
print('Mean Accuracy CV : ', np.mean(cv_results))
print('Std Accuracy CV : ', np.std(cv_results))

Mean Accuracy CV :  0.8379999999999999
Std Accuracy CV :  0.01784656829757476


1) LogisticRegression
2) SGDClassifier
3) LinearSVC

Подготовьте список стоп-слов с помощью `nltk.corpus.stopwords.words('english')` посмотрите на его элементы, и передайте его в соответствующий параметр `CountVectorizer`. 

В `sklearn` также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

In [85]:
stop_words_nltk = set(stopwords.words('english'))

In [86]:
stops = [stop_words_nltk, 'english']
for stop in stops:
    pipeline = Pipeline([
        ('feat_gen', CountVectorizer(stop_words=stop)),
        ('model', LogisticRegression())
    ])

    cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
    print('Mean Accuracy CV : ', np.mean(cv_results))
    print('Std Accuracy CV : ', np.std(cv_results))

Mean Accuracy CV :  0.8400000000000001
Std Accuracy CV :  0.01129158979063624
Mean Accuracy CV :  0.8365
Std Accuracy CV :  0.013472193585307468


Попробуйте в `CountVectorizer` добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра `ngram_range` и параметр `analyzer='char_wb'`. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

In [87]:
# Using bigrams
pipeline = Pipeline([
    ('feat_gen', CountVectorizer(stop_words=stop_words_nltk, ngram_range=(2,2))),
    ('model', LogisticRegression())
])

cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
print('Mean Accuracy CV : ', np.mean(cv_results))
print('Std Accuracy CV : ', np.std(cv_results))

Mean Accuracy CV :  0.764
Std Accuracy CV :  0.008455767262643892


In [88]:
pipeline = Pipeline([
    ('feat_gen', CountVectorizer(stop_words=stop_words_nltk, ngram_range=(3,5), analyzer='char_wb')),
    ('model', LogisticRegression())
])

cv_results = cross_val_score(pipeline, X=all_reviews, y=labels, n_jobs=-1, scoring='accuracy', error_score='raise')
print('Mean Accuracy CV : ', np.mean(cv_results))
print('Std Accuracy CV : ', np.std(cv_results))

Mean Accuracy CV :  0.8255000000000001
Std Accuracy CV :  0.007968688725254608
